In [29]:
import numpy as np
from numba import njit

def sum_array(arr):
    total = 0.0
    for item in arr:
        total += item
    return total

def np_sum_array(arr):
    return np.sum(arr)

def np_arr_diff(arr):
    return arr - arr

def np_arr_np_diff(arr):
    return np.diff(arr)

def np_arr_product(arr):
    return arr * 2 * np.pi

def np_arr_sqrt(arr):
    return np.sqrt(arr)

def np_arr_exp(arr):
    return np.exp(arr)

def np_arr_modulo(arr):
    return arr % 2

def np_clip(arr):
    u = np.clip(arr, 0, 1)

def np_mean(arr):
    return np.mean(arr)

def np_median(arr):
    return np.median(arr)

def np_std(arr):
    return np.std(arr)

def np_abs(arr):
    return np.abs(arr)

def np_sin(arr):
    return np.sin(arr)

def np_cos(arr):
    return np.cos(arr)

def monotonically_increasing(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value


TEST_ARRAY = np.array(range(300))
test_funcs = [
    # These ones are faster in numba
    # sum_array, monotonically_increasing, 
    # np_arr_modulo, np_clip,
    # np_arr_product, np_sum_array,
    np_mean,
    np_std,
    # These ones are not
    # np_arr_diff, np_arr_np_diff,
    # np_arr_exp, np_arr_sqrt,
    np_sin,
    np_cos,
    np_abs,
    ]

def timeit(fn, args):
    import time
    RUNS = 1000000
    fn(args)
    start_time = time.perf_counter()
    for _ in range(RUNS):
        fn(args)
    end_time  = time.perf_counter()
    print((end_time-start_time)/RUNS)

for fn in test_funcs:
    print(f'=============== Testing {fn.__name__} ===============')
    print(f'Non-jitted:')
    timeit(fn, TEST_ARRAY)
    fn = njit(fastmath=True, inline='always')(fn)
    print(f'Jitted:')
    timeit(fn, TEST_ARRAY)


=============== Testing np_mean ===============
Non-jitted:
3.6099028580065352e-06
Jitted:
1.790185930003645e-07
=============== Testing np_std ===============
Non-jitted:
1.1829988487996161e-05
Jitted:
2.8141042700735854e-07
=============== Testing np_sin ===============
Non-jitted:
1.0446081530099037e-06
Jitted:
2.692783676990075e-06
=============== Testing np_cos ===============
Non-jitted:
9.889135880075629e-07
Jitted:
2.7446180219994857e-06
=============== Testing np_abs ===============
Non-jitted:
4.5818364400474823e-07
Jitted:
6.720020799984923e-07


## Summary of Numba Performance
1. `list` could lead to longer exeuction time in JIT optimization, than the non-jitted version. Use `np.array` instead.
2. JIT performs well in: 
    ```
    def np_sum_array(arr):
        return np.sum(arr)
    ```
    - Also performs well in Raw looping in python, where numpy cannot help

3. JIT does NOT perform well (i.e., slower than its non-jitted counter part) in:
    ```
    def np_arr_diff(arr):
        return arr - arr

    def np_arr_product(arr):
        return 3 * arr
    ```

4. USAGE WARNINGS:
    - **Numba does not support class instance methods**. Must use static method / regular function instead
    - **Numba does not support map()**.